In [1]:
!pip install --force-reinstall numpy

  Obtaining dependency information for numpy from https://files.pythonhosted.org/packages/d5/03/0eade211c504bda872a594f045f98ddcc6caef2b7c63610946845e304d3f/numpy-2.3.2-cp311-cp311-win_amd64.whl.metadata
  Using cached numpy-2.3.2-cp311-cp311-win_amd64.whl.metadata (60 kB)
Using cached numpy-2.3.2-cp311-cp311-win_amd64.whl (13.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.2
    Uninstalling numpy-2.3.2:
      Successfully uninstalled numpy-2.3.2


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\evely\\anaconda3\\Lib\\site-packages\\~.mpy.libs\\libscipy_openblas64_-860d95b1c38e637ce4509f5fa24fbf2a.dll'
Consider using the `--user` option or check the permissions.



In [2]:
import torch
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import cv2
import os
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import json
import pickle
from datetime import datetime
import logging

C:\Users\evely\anaconda3\Lib\site-packages\scipy\__init__.py:132: UserWarning: A NumPy version >=1.21.6 and <1.28.0 is required for this version of SciPy (detected version 2.3.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\evely\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\evely\anaconda3\Lib\site-pa

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [1]:
# from accident_detection_model import AccidentDetectionCNN, AccidentDetectionLoss, VideoPreprocessor

class AccidentVideoDataset(Dataset):
    """
    Dataset class for loading accident detection video data
    Expects data structure:
    data_root/
    ├── accidents/
    │   ├── video1.mp4
    │   ├── video2.mp4
    │   └── ...
    ├── normal/
    │   ├── video1.mp4
    │   ├── video2.mp4
    │   └── ...
    └── annotations.json  # Contains frame-level annotations
    """
    def __init__(self, data_root, split='train', sequence_length=16, transform=None):
        self.data_root = data_root
        self.split = split
        self.sequence_length = sequence_length
        self.transform = transform or VideoPreprocessor()
        
        # Load video paths and labels
        self.video_paths, self.labels = self._load_data()
        
        # Load frame-level annotations if available
        self.annotations = self._load_annotations()
        
    def _load_data(self):
        """Load video paths and binary labels"""
        video_paths = []
        labels = []
        
        # Load accident videos (label = 1)
        accident_dir = os.path.join(self.data_root, 'accidents')
        if os.path.exists(accident_dir):
            for video_file in os.listdir(accident_dir):
                if video_file.endswith(('.mp4', '.avi', '.mov')):
                    video_paths.append(os.path.join(accident_dir, video_file))
                    labels.append(1)
        
        # Load normal videos (label = 0)
        normal_dir = os.path.join(self.data_root, 'normal')
        if os.path.exists(normal_dir):
            for video_file in os.listdir(normal_dir):
                if video_file.endswith(('.mp4', '.avi', '.mov')):
                    video_paths.append(os.path.join(normal_dir, video_file))
                    labels.append(0)
        
        return video_paths, labels
    
    def _load_annotations(self):
        """Load detailed frame-level annotations"""
        annotation_path = os.path.join(self.data_root, 'annotations.json')
        if os.path.exists(annotation_path):
            with open(annotation_path, 'r') as f:
                return json.load(f)
        return {}
    
    def _extract_video_frames(self, video_path, max_frames=None):
        """Extract frames from video"""
        cap = cv2.VideoCapture(video_path)
        frames = []
        
        frame_count = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
                
            frames.append(frame)
            frame_count += 1
            
            if max_frames and frame_count >= max_frames:
                break
        
        cap.release()
        return frames
    
    def _create_training_sequences(self, frames, label):
        """Create multiple training sequences from video frames"""
        sequences = []
        
        # Create overlapping sequences
        step_size = self.sequence_length // 2  # 50% overlap
        
        for i in range(0, len(frames) - self.sequence_length + 1, step_size):
            sequence_frames = frames[i:i + self.sequence_length]
            
            # Process sequence
            processed_sequence = self.transform.create_sequence(sequence_frames)
            
            # Create motion intensity labels (higher for accident videos)
            motion_intensity = 0.8 if label == 1 else np.random.uniform(0.1, 0.4)
            
            sequences.append({
                'frames': processed_sequence,
                'anomaly_label': float(label),
                'motion_label': motion_intensity
            })
        
        return sequences
    
    def __len__(self):
        return len(self.video_paths)
    
    def __getitem__(self, idx):
        video_path = self.video_paths[idx]
        label = self.labels[idx]
        
        # Extract frames from video
        frames = self._extract_video_frames(video_path, max_frames=100)
        
        if len(frames) < self.sequence_length:
            # Pad with repeated frames if too short
            frames.extend([frames[-1]] * (self.sequence_length - len(frames)))
        
        # Create training sequences
        sequences = self._create_training_sequences(frames, label)
        
        # Return random sequence for this sample
        selected_sequence = sequences[np.random.randint(0, len(sequences))]
        
        return {
            'frames': torch.FloatTensor(selected_sequence['frames']),
            'anomaly_labels': torch.FloatTensor([selected_sequence['anomaly_label']]),
            'motion_labels': torch.FloatTensor([selected_sequence['motion_label']]),
            'video_path': video_path
        }



C:\Users\evely\anaconda3\Lib\site-packages\scipy\__init__.py:132: UserWarning: A NumPy version >=1.21.6 and <1.28.0 is required for this version of SciPy (detected version 2.3.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\evely\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\evely\anaconda3\Lib\site-pa

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [ ]:
class AccidentDetectionTrainer:
    """
    Training pipeline for accident detection model
    """
    def __init__(self, model, device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.model = model
        self.device = device
        self.model.to(device)
        
        # Initialize loss function and optimizer
        self.loss_fn = AccidentDetectionLoss()
        self.optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
        self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer, patience=5, factor=0.5, verbose=True
        )
        
        # Training history
        self.train_losses = []
        self.val_losses = []
        self.val_accuracies = []
        
        # Setup logging
        self.setup_logging()
    
    def setup_logging(self):
        """Setup logging for training progress"""
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s',
            handlers=[
                logging.FileHandler(f'training_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log'),
                logging.StreamHandler()
            ]
        )
        self.logger = logging.getLogger(__name__)
    
    def train_epoch(self, dataloader):
        """Train for one epoch"""
        self.model.train()
        total_loss = 0
        num_batches = 0
        
        progress_bar = tqdm(dataloader, desc="Training")
        
        for batch in progress_bar:
            # Move data to device
            frames = batch['frames'].to(self.device)
            targets = {
                'anomaly_labels': batch['anomaly_labels'].to(self.device),
                'motion_labels': batch['motion_labels'].to(self.device)
            }
            
            # Forward pass
            self.optimizer.zero_grad()
            outputs = self.model(frames)
            
            # Calculate loss
            loss = self.loss_fn(outputs, targets)
            
            # Backward pass
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
            self.optimizer.step()
            
            # Update metrics
            total_loss += loss.item()
            num_batches += 1
            
            # Update progress bar
            progress_bar.set_postfix({'loss': loss.item()})
        
        avg_loss = total_loss / num_batches
        return avg_loss
    
    def validate_epoch(self, dataloader):
        """Validate for one epoch"""
        self.model.eval()
        total_loss = 0
        predictions = []
        ground_truth = []
        
        with torch.no_grad():
            for batch in tqdm(dataloader, desc="Validation"):
                # Move data to device
                frames = batch['frames'].to(self.device)
                targets = {
                    'anomaly_labels': batch['anomaly_labels'].to(self.device),
                    'motion_labels': batch['motion_labels'].to(self.device)
                }
                
                # Forward pass
                outputs = self.model(frames)
                loss = self.loss_fn(outputs, targets)
                
                # Store predictions and ground truth
                predictions.extend((outputs['anomaly_score'] > 0.5).cpu().numpy().flatten())
                ground_truth.extend(targets['anomaly_labels'].cpu().numpy().flatten())
                
                total_loss += loss.item()
        
        # Calculate metrics
        avg_loss = total_loss / len(dataloader)
        accuracy = accuracy_score(ground_truth, predictions)
        precision, recall, f1, _ = precision_recall_fscore_support(
            ground_truth, predictions, average='binary'
        )
        
        return {
            'loss': avg_loss,
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1
        }
    
    def train(self, train_loader, val_loader, num_epochs=50, save_path='best_model.pth'):
        """Full training loop"""
        best_val_accuracy = 0
        patience_counter = 0
        max_patience = 10
        
        self.logger.info(f"Starting training for {num_epochs} epochs")
        self.logger.info(f"Device: {self.device}")
        self.logger.info(f"Model parameters: {sum(p.numel() for p in self.model.parameters()):,}")
        
        for epoch in range(num_epochs):
            self.logger.info(f"\nEpoch {epoch + 1}/{num_epochs}")
            
            # Train
            train_loss = self.train_epoch(train_loader)
            self.train_losses.append(train_loss)
            
            # Validate
            val_metrics = self.validate_epoch(val_loader)
            self.val_losses.append(val_metrics['loss'])
            self.val_accuracies.append(val_metrics['accuracy'])
            
            # Learning rate scheduling
            self.scheduler.step(val_metrics['loss'])
            
            # Log metrics
            self.logger.info(f"Train Loss: {train_loss:.4f}")
            self.logger.info(f"Val Loss: {val_metrics['loss']:.4f}")
            self.logger.info(f"Val Accuracy: {val_metrics['accuracy']:.4f}")
            self.logger.info(f"Val Precision: {val_metrics['precision']:.4f}")
            self.logger.info(f"Val Recall: {val_metrics['recall']:.4f}")
            self.logger.info(f"Val F1: {val_metrics['f1']:.4f}")
            
            # Save best model
            if val_metrics['accuracy'] > best_val_accuracy:
                best_val_accuracy = val_metrics['accuracy']
                patience_counter = 0
                
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': self.model.state_dict(),
                    'optimizer_state_dict': self.optimizer.state_dict(),
                    'val_accuracy': val_metrics['accuracy'],
                    'train_history': {
                        'train_losses': self.train_losses,
                        'val_losses': self.val_losses,
                        'val_accuracies': self.val_accuracies
                    }
                }, save_path)
                
                self.logger.info(f"New best model saved! Accuracy: {best_val_accuracy:.4f}")
            else:
                patience_counter += 1
            
            # Early stopping
            if patience_counter >= max_patience:
                self.logger.info(f"Early stopping triggered after {epoch + 1} epochs")
                break
        
        self.logger.info(f"Training completed. Best validation accuracy: {best_val_accuracy:.4f}")
        return best_val_accuracy
    
    def plot_training_history(self):
        """Plot training history"""
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        # Loss plot
        ax1.plot(self.train_losses, label='Train Loss', color='blue')
        ax1.plot(self.val_losses, label='Validation Loss', color='red')
        ax1.set_title('Training and Validation Loss')
        ax1.set_xlabel('Epoch')
        ax1.set_ylabel('Loss')
        ax1.legend()
        ax1.grid(True)
        
        # Accuracy plot
        ax2.plot(self.val_accuracies, label='Validation Accuracy', color='green')
        ax2.set_title('Validation Accuracy')
        ax2.set_xlabel('Epoch')
        ax2.set_ylabel('Accuracy')
        ax2.legend()
        ax2.grid(True)
        
        plt.tight_layout()
        plt.savefig('training_history.png', dpi=300, bbox_inches='tight')
        plt.show()



In [ ]:
class AccidentDetectionEvaluator:
    """
    Evaluation pipeline for accident detection model
    """
    def __init__(self, model, device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.model = model
        self.device = device
        self.model.to(device)
    
    def evaluate_model(self, test_loader, threshold=0.5):
        """Comprehensive model evaluation"""
        self.model.eval()
        
        all_predictions = []
        all_probabilities = []
        all_ground_truth = []
        all_motion_predictions = []
        
        with torch.no_grad():
            for batch in tqdm(test_loader, desc="Evaluating"):
                frames = batch['frames'].to(self.device)
                labels = batch['anomaly_labels'].cpu().numpy()
                
                outputs = self.model(frames)
                probabilities = outputs['anomaly_score'].cpu().numpy()
                predictions = (probabilities > threshold).astype(int)
                motion_pred = outputs['motion_intensity'].cpu().numpy()
                
                all_predictions.extend(predictions.flatten())
                all_probabilities.extend(probabilities.flatten())
                all_ground_truth.extend(labels.flatten())
                all_motion_predictions.extend(motion_pred.flatten())
        
        # Calculate comprehensive metrics
        accuracy = accuracy_score(all_ground_truth, all_predictions)
        precision, recall, f1, _ = precision_recall_fscore_support(
            all_ground_truth, all_predictions, average='binary'
        )
        
        # Confusion matrix
        cm = confusion_matrix(all_ground_truth, all_predictions)
        
        results = {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'confusion_matrix': cm,
            'predictions': all_predictions,
            'probabilities': all_probabilities,
            'ground_truth': all_ground_truth,
            'motion_predictions': all_motion_predictions
        }
        
        return results
    
    def plot_evaluation_results(self, results):
        """Plot evaluation results"""
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
        
        # Confusion Matrix
        sns.heatmap(results['confusion_matrix'], annot=True, fmt='d', ax=ax1, 
                   xticklabels=['Normal', 'Accident'], 
                   yticklabels=['Normal', 'Accident'])
        ax1.set_title('Confusion Matrix')
        ax1.set_xlabel('Predicted')
        ax1.set_ylabel('Actual')
        
        # ROC Curve (simplified)
        from sklearn.metrics import roc_curve, auc
        fpr, tpr, _ = roc_curve(results['ground_truth'], results['probabilities'])
        roc_auc = auc(fpr, tpr)
        
        ax2.plot(fpr, tpr, color='darkorange', lw=2, 
                label=f'ROC curve (AUC = {roc_auc:.2f})')
        ax2.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        ax2.set_xlabel('False Positive Rate')
        ax2.set_ylabel('True Positive Rate')
        ax2.set_title('ROC Curve')
        ax2.legend()
        ax2.grid(True)
        
        # Probability Distribution
        ax3.hist([p for i, p in enumerate(results['probabilities']) if results['ground_truth'][i] == 0], 
                alpha=0.7, label='Normal', bins=30)
        ax3.hist([p for i, p in enumerate(results['probabilities']) if results['ground_truth'][i] == 1], 
                alpha=0.7, label='Accident', bins=30)
        ax3.set_xlabel('Anomaly Score')
        ax3.set_ylabel('Frequency')
        ax3.set_title('Score Distribution')
        ax3.legend()
        ax3.grid(True)
        
        # Metrics Summary
        metrics_text = f"""
        Accuracy: {results['accuracy']:.3f}
        Precision: {results['precision']:.3f}
        Recall: {results['recall']:.3f}
        F1-Score: {results['f1_score']:.3f}
        AUC: {roc_auc:.3f}
        """
        ax4.text(0.1, 0.5, metrics_text, fontsize=12, verticalalignment='center')
        ax4.axis('off')
        ax4.set_title('Performance Metrics')
        
        plt.tight_layout()
        plt.savefig('evaluation_results.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        return results



In [ ]:
# Example usage
if __name__ == "__main__":
    # Data paths (update these to your actual data paths)
    DATA_ROOT = "/path/to/your/accident/dataset"
    
    # Create datasets
    print("Loading datasets...")
    train_dataset = AccidentVideoDataset(DATA_ROOT, split='train')
    val_dataset = AccidentVideoDataset(DATA_ROOT, split='val')
    test_dataset = AccidentVideoDataset(DATA_ROOT, split='test')
    
    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=2)
    
    print(f"Train samples: {len(train_dataset)}")
    print(f"Validation samples: {len(val_dataset)}")
    print(f"Test samples: {len(test_dataset)}")
    
    # Initialize model
    model = AccidentDetectionCNN(num_classes=1, sequence_length=16)
    
    # Initialize trainer
    trainer = AccidentDetectionTrainer(model)
    
    # Train the model
    print("\nStarting training...")
    best_accuracy = trainer.train(
        train_loader=train_loader,
        val_loader=val_loader,
        num_epochs=30,
        save_path='best_accident_model.pth'
    )
    
    # Plot training history
    trainer.plot_training_history()
    
    # Load best model for evaluation
    checkpoint = torch.load('best_accident_model.pth')
    model.load_state_dict(checkpoint['model_state_dict'])
    
    # Evaluate on test set
    evaluator = AccidentDetectionEvaluator(model)
    test_results = evaluator.evaluate_model(test_loader)
    
    # Plot evaluation results
    evaluator.plot_evaluation_results(test_results)
    
    print(f"\nFinal Test Results:")
    print(f"Accuracy: {test_results['accuracy']:.4f}")
    print(f"Precision: {test_results['precision']:.4f}")
    print(f"Recall: {test_results['recall']:.4f}")
    print(f"F1-Score: {test_results['f1_score']:.4f}")
    
    # Save test results
    with open('test_results.json', 'w') as f:
        # Convert numpy arrays to lists for JSON serialization
        serializable_results = {
            'accuracy': float(test_results['accuracy']),
            'precision': float(test_results['precision']),
            'recall': float(test_results['recall']),
            'f1_score': float(test_results['f1_score']),
            'confusion_matrix': test_results['confusion_matrix'].tolist()
        }
        json.dump(serializable_results, f, indent=2)
    
    print("\nTraining and evaluation complete!")
    print("Results saved to test_results.json")
    print("Plots saved as training_history.png and evaluation_results.png")